In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy


PATH='/home/wk/myProjects/data/Enron/oper/'

TRN_PATH = 'train/'
VAL_PATH = 'test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

bs=32; bptt=75
em_sz = 300  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

learner.load_encoder('adam3_10_enc')

In [2]:
#PATH=Path('/home/wk/myProjects/data/Enron/tag/')

In [3]:
#[str(i) for i in PATH.iterdir()]
#PATH/'here.txt'.open()

In [4]:
### Test
ss=r"""
Message-ID: <5668764.1075845986267.JavaMail.evans@thyme>
Date: Thu, 15 Feb 2001 06:30:00 -0800 (PST)
From: kay.mann@enron.com
To: kathleen.carnahan@enron.com
Subject: Re: Form CA for selling power projects
Cc: carlos.sole@enron.com
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
Bcc: carlos.sole@enron.com
X-From: Kay Mann
X-To: Kathleen Carnahan
X-cc: Carlos Sole
X-bcc: 
X-Folder: \Kay_Mann_June2001_3\Notes Folders\Sent
X-Origin: MANN-K
X-FileName: kmann.nsf

Yes.


   
	
	
	From:  Kathleen Carnahan                           02/15/2001 02:04 PM
	

To: Kay Mann/Corp/Enron@Enron
cc: Carlos Sole/NA/Enron@Enron 

Subject: Re: Form CA for selling power projects  

And so am I!  Did you add your changes to it yet?

Kathleen



Kay Mann
02/15/2001 02:02 PM
To: Kathleen Carnahan/NA/Enron@Enron, Carlos Sole/NA/Enron@Enron
cc:  

Subject: Form CA for selling power projects

I set up a directory called "Confidentiality Agreements" and put a form in 
there based on what KC sent me.

I'm so proud of myself.




"""
s = [TEXT.preprocess(ss)] 
t=TEXT.numericalize(s)
' '.join(s[0])

'message-id: <5668764.1075845986267.javamail.evans@thyme> date: thu, 15 feb 2001 06:30:00 -0800 (pst) from: kay.mann@enron.com to: kathleen.carnahan@enron.com subject: re: form ca for selling power projects cc: carlos.sole@enron.com mime-version: 1.0 content-type: text/plain; charset=us-ascii content-transfer-encoding: 7bit bcc: carlos.sole@enron.com x-from: kay mann x-to: kathleen carnahan x-cc: carlos sole x-bcc: x-folder: \\kay_mann_june2001_3\\notes folders\\sent x-origin: mann-k x-filename: kmann.nsf yes. from: kathleen carnahan 02/15/2001 02:04 pm to: kay mann/corp/enron@enron cc: carlos sole/na/enron@enron subject: re: form ca for selling power projects and so am i! did you add your changes to it yet? kathleen kay mann 02/15/2001 02:02 pm to: kathleen carnahan/na/enron@enron, carlos sole/na/enron@enron cc: subject: form ca for selling power projects i set up a directory called "confidentiality agreements" and put a form in there based on what kc sent me. i\'m so proud of myself.

In [5]:
import copy
ss = copy.deepcopy(s)
s_p = [ss[0][0:3]]
t=TEXT.numericalize(s)
#t = t.long().cpu()

m=learner.model

m[0].bs=1
m.eval()
m.reset()

#m.cpu()
#m.cuda()
res,*_ = m(t)

nexts = torch.topk(res[-1], 10)[1]

for i in range(len(s[0])-3):
    n=res[-1].topk(2)[1]    
    predict = TEXT.vocab.itos[n.data[0]]    
    if (predict[0:2] == '@@'):
        print("pos=" + str(i+3) + "; " + predict + " <- " + s[0][i+3] , end=' \n')                    
        ss[0][i+3]=predict
    
    s_p = [ss[0][0:i+4]]
    t=TEXT.numericalize(s_p)

    res,*_ = m(t)    
    
m[0].bs=bs       
print('...')

pos=11; @@email@@ <- kay.mann@enron.com 
pos=34; @@firstname@@ <- kay 
pos=35; @@lastname@@ <- mann 
pos=58; @@firstname@@ <- kay 
pos=59; @@lastname@@ <- mann/corp/enron@enron 
pos=71; @@firstname@@, <- and 
pos=84; @@firstname@@ <- kay 
pos=85; @@lastname@@ <- mann 
...


In [6]:
#https://askubuntu.com/questions/607118/cuda-not-working-after-returning-laptop-from-sleep#750939
len(s[0])1

SyntaxError: invalid syntax (<ipython-input-6-0979a38a4286>, line 2)

In [ ]:
#sudo rmmod nvidia_uvm
#sudo modprobe nvidia_uvm

In [ ]:
#t.long().cpu()